In [ ]:
# -*- coding: utf-8 -*-
"""
Projeto de Machine Learning: Classificação de Nível de Alerta de Incêndio
Autora: Tarciane Samily
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import joblib

# -----------------------------
# 1. Gerar dados sintéticos
# -----------------------------
np.random.seed(42)
n = 200
countries = ['Brazil', 'USA', 'Australia', 'Canada', 'Spain']
alert_levels = ['GREEN', 'YELLOW', 'RED', 'ORANGE']

data = {
    'country': np.random.choice(countries, n),
    'severity': np.random.randint(1, 101, n),
    'Duration (days)': np.random.randint(1, 15, n),
    'alertlevel': np.random.choice(alert_levels, n)
}

df = pd.DataFrame(data)

# Salvar CSV
df.to_csv('Forest_Fires.csv', index=False)
print("Arquivo Forest_Fires.csv criado com sucesso!")
print(df.head())

# -----------------------------
# 2. Pré-processamento e criação da variável alvo
# -----------------------------
# Transformar alertlevel para maiúsculas e remover ORANGE
df['alertlevel'] = df['alertlevel'].str.upper()
df = df[df['alertlevel'] != 'ORANGE']

# Criar variável alvo binária
median_severity = df['severity'].median()
df['alert_binary'] = df['severity'].apply(lambda x: 'HIGH' if x > median_severity else 'LOW')
print(f"\nMediana de severidade usada para criar alert_binary: {median_severity}")
print(df['alert_binary'].value_counts())

# Selecionar features e target
features = ['country', 'severity', 'Duration (days)']
target = 'alert_binary'
X = df[features].values
y = df[target].values

# -----------------------------
# 3. Pré-processamento das features
# -----------------------------
cat_index = 0  # coluna categórica 'country'

preprocessor = Pipeline([
    ('onehot', ColumnTransformer(
        transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), [cat_index])],
        remainder='passthrough'
    )),
    ('scaler', StandardScaler(with_mean=False))
])

X_processed = preprocessor.fit_transform(X)

# Codificar variável alvo
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
class_names = label_encoder.classes_

print("\nDimensões após pré-processamento:", X_processed.shape)
print("Classes do alvo:", class_names)

# -----------------------------
# 4. Divisão treino/teste
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_encoded, test_size=0.2, random_state=42
)
print("\nDimensões do conjunto de treino:", X_train.shape)
print("Dimensões do conjunto de teste:", X_test.shape)

# -----------------------------
# 5. Treinar Decision Tree
# -----------------------------
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
print("\nDecision Tree treinada com sucesso!")

# -----------------------------
# 6. Avaliação do modelo
# -----------------------------
y_pred = dt_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"\nAcurácia do modelo: {accuracy*100:.2f}%")

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title("Matriz de Confusão - Decision Tree")
plt.xlabel("Predito")
plt.ylabel("Verdadeiro")
plt.show()

# -----------------------------
# 7. Salvar modelo e preprocessor
# -----------------------------
joblib.dump(dt_model, "decision_tree_model.pkl")
joblib.dump(preprocessor, "preprocessor.pkl")
print("\nModelo e preprocessor salvos com sucesso!")



Taxa de Churn (cancelaram): 43.33%
Taxa de Retenção (não cancelaram): 56.67%
